In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib import patches
plt.ion()

import time
from UDPComms import Subscriber
import math

In [2]:
def box_plot():
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_xlim(0, 600)
    ax.set_ylim(-10, 10)
    
    r = patches.Rectangle((0, 0), 10, 1, fc='r', alpha=1)
    g = patches.Rectangle((0, 0), 10, 1, fc='b', alpha=0.5)
    b = patches.Rectangle((0, 0), 10, 1, fc='g', alpha=0.5)
    ax.add_patch(r)
    ax.add_patch(g)
    ax.add_patch(b)
    fig.show()

    def update(y,c="r"):
            if c== "r":
                r.set_x(y)
            elif c=="b":
                b.set_x(y)
            elif c=="g":
                g.set_x(y)
            else:
                raise ValueError
            fig.canvas.draw()
    return update

In [3]:
def liveplot():
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    red_data = []
    green_data = []
    blue_data = []

    fig.show()

    def update(y,c="r"):
            if c== "r":
                red_data.append(y)
            elif c=="b":
                blue_data.append(y)
            elif c=="g":
                green_data.append(y)
            else:
                raise ValueError

            ax.clear()
            ax.plot(red_data, 'r-')
            ax.plot(blue_data, 'b-')
            ax.plot(green_data, 'g-')
            fig.canvas.draw()
    return update

In [4]:
sim = Subscriber(8810, timeout=0.5)

In [11]:
update = liveplot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
intergrated = sim.recv()['dist']
rate = 0.02

# raw readings

while 1:
    start = time.time()
    msg = sim.get()
    
    update(msg['truth'], 'r')
    update(msg['dist'], 'b')
    update(intergrated, 'g')
    time.sleep(rate)
    
    intergrated += (time.time()-start) * msg['vel']

KeyboardInterrupt: 

In [22]:
update = liveplot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
mean = sim.recv()['dist']
rate = 0.01
alpha = 0.5


# low pass filter readings

while 1:
    start = time.time()
    msg = sim.get()
    
    update(msg['truth'], 'r')
    update(msg['dist'], 'g')
    
    # low pass filter
    mean = mean * alpha + msg['dist'] * (1-alpha)
    update(mean, 'b')

    time.sleep(rate)

KeyboardInterrupt: 

In [24]:
update = liveplot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
mean = sim.recv()['dist']
rate = 0.05
alpha = 0.9

# complimentary filter

while 1:
    start = time.time()
    msg = sim.get()
    
    update(msg['truth'], 'r')
    update(msg['dist'], 'b')
    
    # low pass filter
    mean = mean * alpha + msg['dist'] * (1-alpha)   
    update(mean, 'g')

    time.sleep(rate)
    mean = mean + (time.time()-start) * msg['vel']

KeyboardInterrupt: 

In [26]:
def liveplot_confidence():
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    truth_data = []
    mean_data = []
    std_data = []


    fig.show()

    def update(t,m,s):
            truth_data.append(t)
            mean_data.append(m)
            std_data.append(s)

            top = [a+2*b for a,b in zip(mean_data, std_data)]
            bottom = [a-2*b for a,b in zip(mean_data, std_data)]

            ax.clear()
            ax.plot(truth_data, 'r-')
            ax.plot(mean_data, 'b-')
            ax.fill_between( list(range(len(top))), top,bottom, facecolor='y',alpha = 0.5)
            fig.canvas.draw()
    return update

In [31]:
update = liveplot_confidence()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
mean = sim.recv()['dist']
sigma = 100
rate = 0.05

vel_std = 30
pos_std = 40

# kalman filter
dt = rate
while 1:
    start = time.time()
    msg = sim.get()
    
    msg['dist']
    msg['vel']
    
    # prediction step
    mean += dt * msg['vel']
    sigma += dt* vel_std
    
    # measurement step
    K = sigma/(sigma + pos_std)
    mean += K*(msg['dist'] - mean)
    sigma -= K*sigma

    update(msg['truth'], mean, sigma)

    time.sleep(rate)
    dt = time.time()-start

KeyboardInterrupt: 

In [33]:
# issues on manifolds

imu = Subscriber(8007, 0.5)

In [38]:
update=liveplot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
mean = 0
rate = 0.05
alpha = 0.6


# low pass filter readings

while 1:
    start = time.time()
    ax,ay,az, gx, gy, gz = imu.get()
    
    angle = math.degrees(math.atan2(ay, ax))
    
    update(angle, 'r')
    update(mean, 'g')
    
    # low pass filter
    mean = mean * alpha + angle * (1-alpha)

    time.sleep(rate)

KeyboardInterrupt: 